# **Installing Libs**

In [ ]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 30.3 MB/s eta 0:00:00


In [ ]:
pip install gmaps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076084 sha256=8f7fc9053a6877c106f5a0e800b3f9ead20b73b765e8a3b7b6768b6cea7df898
  Stored in directory: /root/.cache/pip/wheels/b3/c2/dc/48b3ef16c2184dae51a003f17eb5d065bbbf1af3437d9f14e3
Successfully built gmaps


In [ ]:
pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40712 sha256=f73d4520051974b53a59610d917948feea451ade241c609e394453233137a7a7
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 24.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.4.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.2

# **1st Stage**

In [ ]:
import collections
collections.Iterable = collections.abc.Iterable

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pulp
import itertools
import gmaps
import googlemaps

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [ ]:
API_KEY = 'API'
gmaps.configure(api_key=API_KEY)
googlemaps = googlemaps.Client(key=API_KEY)

In [ ]:
# customer count ('0' is depot)
customer_count = 5

# the number of vehicle
vehicle_count = 4

# the capacity of vehicle
vehicle_capacity = 50

# fix random seed
np.random.seed(seed=277)

# set depot latitude and longitude
# South Tangerang
depot_latitude = -6.2741
depot_longitude = 106.7206
# New York
# depot_latitude = 40.748817
# depot_longitude = -73.985428

# make dataframe which contains vending machine location and demand
df = pd.DataFrame({"latitude":np.random.normal(depot_latitude, 0.07, customer_count),
                   "longitude":np.random.normal(depot_longitude, 0.07, customer_count),
                   "demand":np.random.randint(1, 10, customer_count)})

df.loc[0, 'latitude'] = depot_latitude
df.loc[0, 'longitude'] = depot_longitude
df.loc[0, 'demand'] = 0

df

,latitude,longitude,demand
0,-6.274100,106.720600,0
1,-6.286941,106.762573,1
2,-6.262955,106.794014,8
3,-6.306317,106.704369,8
4,-6.230505,106.871689,7


In [ ]:
# function for plotting on google maps
def _plot_on_gmaps(_df):

    _marker_locations = []
    for i in range(len(_df)):
        _marker_locations.append((_df['latitude'].iloc[i],_df['longitude'].iloc[i]))

    _fig = gmaps.figure()
    _markers = gmaps.marker_layer(_marker_locations,label = [str(item) for item in list(_df.index)])
    _fig.add_layer(_markers)

    return _fig

# function for calculating distance between two pins
def _distance_calculator(_df):

    _distance_result = np.zeros((len(_df),len(_df)))
    _df['latitude-longitude'] = '0'
    for i in range(len(_df)):
        _df['latitude-longitude'].iloc[i] = str(_df.latitude[i]) + ',' + str(_df.longitude[i])

    for i in range(len(_df)):
        for j in range(len(_df)):

            # calculate distance of all pairs
            _google_maps_api_result = googlemaps.directions(_df['latitude-longitude'].iloc[i],
                                                            _df['latitude-longitude'].iloc[j],
                                                            mode = 'driving')
            # append distance to result list
            _distance_result[i][j] = _google_maps_api_result[0]['legs'][0]['distance']['value']

    return _distance_result

In [ ]:
distance = _distance_calculator(df)
plot_result = _plot_on_gmaps(df)
plot_result

<ipython-input-28-87539e57a28f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['latitude-longitude'].iloc[i] = str(_df.latitude[i]) + ',' + str(_df.longitude[i])
<ipython-input-28-87539e57a28f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['latitude-longitude'].iloc[i] = str(_df.latitude[i]) + ',' + str(_df.longitude[i])
<ipython-input-28-87539e57a28f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['latitude-longi

Figure(layout=FigureLayout(height='420px'))

In [ ]:
data = {}
data["distance_matrix"] = distance.astype(int)
data["demands"] = list(df.demand)
data["vehicle_capacity"] = [25,25,25,25]
data["num_vehicles"] = 4
data["depot"] = 0

In [ ]:
data

{'distance_matrix': array([[    0,  8053, 18086,  8361, 33892],
        [ 8825,     0,  6884, 11702, 23938],
        [15008,  6777,     0, 21719, 13566],
        [ 8822, 10475, 16646,     0, 42189],
        [36349, 25420, 13133, 43060,     0]]),
 'demands': [0, 1, 8, 8, 7],
 'vehicle_capacity': [25, 25, 25, 25],
 'num_vehicles': 4,
 'depot': 0}

In [ ]:
manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

In [ ]:
routing = pywrapcp.RoutingModel(manager)

In [ ]:
def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

In [ ]:
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [ ]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
routing.SetAllowedVehiclesForIndex([0], manager.NodeToIndex(1))

In [ ]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data["demands"][from_node]

In [ ]:
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacity"],  # vehicle maximum capacities
        False,  # start cumul to zero
        "Capacity",
)

True

In [ ]:
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    20000,  # vehicle maximum travel distance
    False,  # start cumul to zero
    dimension_name
)

True

In [ ]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    route = {}
    for vehicle_id in range(data["num_vehicles"]):
        route[vehicle_id] = []
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
            route[vehicle_id].append(node_index)
        route[vehicle_id].append(0)
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}"
        print(plan_output)
        print("max load: ",data["vehicle_capacity"][vehicle_id],"\n")
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")

    return route

In [ ]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)
search_parameters.time_limit.FromSeconds(1)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    route = print_solution(data, manager, routing, solution)

In [ ]:
route_clean = route.copy()
keys_to_delete = []
for key, value in route_clean.items():
    if value == [0, 0]:
        keys_to_delete.append(key)

for key in keys_to_delete:
    del route_clean[key]
route_clean

{0: [0, 2, 4, 1, 3, 0]}

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

fig = gmaps.figure()
layer = []
color_list = ["red","blue","green","#7FFFD4"]
for i in route_clean.keys():
  for j in range(len(route_clean[i])-1):
        layer.append(gmaps.directions.Directions(
            (df.latitude[route_clean[i][j]],df.longitude[route_clean[i][j]]),
            (df.latitude[route_clean[i][j+1]],df.longitude[route_clean[i][j+1]]),
            mode='car',stroke_color=color_list[i],stroke_opacity=1.0, stroke_weight=5.0))
for i in range(len(layer)):
  fig.add_layer(layer[i])
fig

Figure(layout=FigureLayout(height='420px'))

# **2nd Stage**

In [ ]:
route_clean

{0: [0, 1, 4, 15, 11, 7, 6, 0],
 1: [0, 2, 17, 9, 18, 8, 0],
 2: [0, 3, 10, 19, 5, 0],
 3: [0, 12, 13, 16, 14, 0]}

In [ ]:
route_clean_new = route_clean.copy()

In [ ]:
perc = 0.3
remove_node0 = []
for key in list(route_clean_new.keys()):
  route_clean_new[key] = route_clean[key][round(len(route_clean[key])*perc):]
  remove_node0.append(route_clean[key][1:round(len(route_clean[key])*perc)])

remove_node = []
[remove_node.extend(sublist) for sublist in remove_node0]

route_clean_new

{0: [4, 15, 11, 7, 6, 0],
 1: [17, 9, 18, 8, 0],
 2: [10, 19, 5, 0],
 3: [13, 16, 14, 0]}

In [ ]:
remove_node

[1, 2, 3, 12]

In [ ]:
df2 = pd.DataFrame({"latitude":np.random.normal(depot_latitude, 0.007, customer_count),
                   "longitude":np.random.normal(depot_longitude, 0.007, customer_count),
                   "demand":np.random.randint(1, 10, customer_count)})



In [ ]:
df_new = pd.concat([df, df2], axis=0,ignore_index=True)

In [ ]:
df_new

,latitude,longitude,demand,latitude-longitude
0,-6.274100,106.720600,0,"-6.2741,106.7206"
1,-6.275384,106.721725,6,"-6.275384106421503,106.72172537685795"
2,-6.272985,106.723493,4,"-6.272985479665504,106.72349298803398"
3,-6.277322,106.727716,7,"-6.2773217016580265,106.72771594944021"
4,-6.269740,106.722429,6,"-6.269740478901571,106.72242855731476"
5,-6.280665,106.725164,4,"-6.280664841863825,106.7251641979343"
6,-6.269903,106.715015,4,"-6.2699027003677665,106.71501486946045"
7,-6.266759,106.713001,5,"-6.266758632314382,106.71300133032842"
8,-6.275723,106.728241,3,"-6.275723102603476,106.72824077573286"
9,-6.258991,106.722276,9,"-6.2589910579843515,106.72227562048162"


In [ ]:
distance_new = _distance_calculator(df_new)

<ipython-input-406-87539e57a28f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['latitude-longitude'].iloc[i] = str(_df.latitude[i]) + ',' + str(_df.longitude[i])
<ipython-input-406-87539e57a28f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['latitude-longitude'].iloc[i] = str(_df.latitude[i]) + ',' + str(_df.longitude[i])
<ipython-input-406-87539e57a28f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['latitude-lo

In [ ]:
data_new = {}
data_new["distance_matrix"] = distance_new.astype(int)
data_new["demands"] = list(df_new.demand)
data_new["num_vehicles"] = 4
data_new["depot"] = 0
data_new["vehicle_capacity"] = [250,250,250,250]
data_new['starts'] = []
data_new['ends'] = []
for key in list(route_clean_new.keys()):
  data_new['starts'].append(route_clean_new[key][0])
  data_new['ends'].append(0)
data_new

{'distance_matrix': array([[   0,  295, 1239, ..., 2439, 2533, 1240],
        [ 295,    0,  944, ..., 2144, 2238,  945],
        [2297, 2002,    0, ..., 1266, 1360, 1469],
        ...,
        [3362, 3067, 1993, ...,    0,  454, 2534],
        [3506, 3211, 2137, ...,  503,    0, 2678],
        [1936, 1641, 1312, ..., 2512, 2606,    0]]),
 'demands': [0,
  6,
  4,
  7,
  6,
  4,
  4,
  5,
  3,
  9,
  9,
  1,
  6,
  7,
  4,
  3,
  8,
  4,
  5,
  5,
  1,
  9,
  4,
  8,
  1,
  6,
  2,
  6,
  2,
  7,
  2,
  6,
  3,
  3,
  2,
  4,
  5,
  5,
  6,
  4],
 'num_vehicles': 4,
 'depot': 0,
 'vehicle_capacity': [250, 250, 250, 250],
 'starts': [4, 17, 10, 13],
 'ends': [0, 0, 0, 0]}

In [ ]:
for node in remove_node:
  data_new['demands'][node] = 1000000

In [ ]:
manager = pywrapcp.RoutingIndexManager(
        len(data_new["distance_matrix"]), data_new["num_vehicles"], data_new["starts"], data_new["ends"]
    )

In [ ]:
routing = pywrapcp.RoutingModel(manager)

In [ ]:
# avoiding nodes that already visited
penalty = 100000000000
for node in remove_node:
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

In [ ]:
def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data_new["distance_matrix"][from_node][to_node]
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [ ]:
# vehicle k must visit node i
for key in list(route_clean_new.keys()):
  for idx in route_clean_new[key][:-1]:
    routing.SetAllowedVehiclesForIndex([key], manager.NodeToIndex(idx))

In [ ]:
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    18000,  # vehicle maximum travel distance
    False,  # start cumul to zero
    dimension_name
)

True

In [ ]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data_new["demands"][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data_new["vehicle_capacity"],  # vehicle maximum capacities
        False,  # start cumul to zero
        "Capacity",
)

True

In [ ]:
def print_solution2(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    #total_load = 0
    route = {}
    for vehicle_id in range(data["num_vehicles"]):
        route[vehicle_id] = []
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        #route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            #route_load += data["demands"][node_index]
            plan_output += f" {node_index} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
            route[vehicle_id].append(node_index)
        route[vehicle_id].append(0)
        plan_output += f" {manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        #plan_output += f"Load of the route: {route_load}"
        print(plan_output)
        #print("max load: ",data["vehicle_capacity"][vehicle_id],"\n")
        total_distance += route_distance
        #total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    #print(f"Total load of all routes: {total_load}")

    return route

In [ ]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)
search_parameters.time_limit.FromSeconds(1)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    route = print_solution2(data_new, manager, routing, solution)

Objective: 400000045912
Route for vehicle 0:
 4 ->  39 ->  15 ->  11 ->  29 ->  27 ->  7 ->  6 ->  28 ->  0
Distance of the route: 14982m

Route for vehicle 1:
 17 ->  9 ->  22 ->  31 ->  33 ->  24 ->  30 ->  32 ->  18 ->  8 ->  36 ->  0
Distance of the route: 12248m

Route for vehicle 2:
 10 ->  26 ->  23 ->  21 ->  19 ->  20 ->  5 ->  35 ->  0
Distance of the route: 7603m

Route for vehicle 3:
 13 ->  16 ->  14 ->  34 ->  25 ->  38 ->  37 ->  0
Distance of the route: 11079m

Total distance of all routes: 45912m


In [ ]:
route_clean = route.copy()
keys_to_delete = []
for key, value in route_clean.items():
    if value == [0, 0]:
        keys_to_delete.append(key)

for key in keys_to_delete:
    del route_clean[key]
route_clean

{0: [4, 39, 15, 11, 29, 27, 7, 6, 28, 0],
 1: [17, 9, 22, 31, 33, 24, 30, 32, 18, 8, 36, 0],
 2: [10, 26, 23, 21, 19, 20, 5, 35, 0],
 3: [13, 16, 14, 34, 25, 38, 37, 0]}

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

fig = gmaps.figure()
layer = []
color_list = ["red","blue","green","#7FFFD4"]
for i in route_clean.keys():
  for j in range(len(route_clean[i])-1):
        layer.append(gmaps.directions.Directions(
            (df_new.latitude[route_clean[i][j]],df_new.longitude[route_clean[i][j]]),
            (df_new.latitude[route_clean[i][j+1]],df_new.longitude[route_clean[i][j+1]]),
            mode='car',stroke_color=color_list[i],stroke_opacity=1.0, stroke_weight=5.0))
for i in range(len(layer)):
  fig.add_layer(layer[i])
fig

Figure(layout=FigureLayout(height='420px'))